#### Mount the google drive to import the model configuration

In [ ]:
# to mount the drive
def mountDrive(force_remount=True):
    from google.colab import drive
    print('drive_filepath="drive/My Drive/"')
    return drive.mount('/content/drive', force_remount=force_remount)

mountDrive()

drive_filepath="drive/My Drive/"
Mounted at /content/drive


#### Scraping the privacy policy

In [ ]:
%%capture
!pip install selenium
!pip install lxml
!apt-get update 
!apt install chromium-chromedriver

!pip install nltk

!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
#import nltk
#from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
#import numpy as np
import networkx as nx
#import pandas as pd
#from google.colab import drive
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
%%capture
##libraries used

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

#import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import brown
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('brown')


#import pandas as pd
#import re
import string
import os
import numpy as np
import torch
import transformers
from datasets import load_dataset, Dataset, load_from_disk
#from google.colab import drive
#import csv

from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
#from transformers import DataCollatorWithPadding
#from transformers import create_optimizer
from datasets import load_metric
#import datasets
#from transformers.keras_callbacks import KerasMetricCallback

#import nltk
#from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize
#import numpy as np
import networkx as nx
#import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# # to mount the drive

# def mountDrive(force_remount=True):
#     from google.colab import drive
#     print('drive_filepath="drive/My Drive/"')
#     return drive.mount('/content/drive', force_remount=force_remount)

In [ ]:
# to scrape the policy using headless browser and beautiful soup

def scrapeUrl(url):
  #url = "https://www.whatsapp.com/legal/privacy-policy"
  
  options = Options()
  options.add_argument("--window-size=1920,1200")
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  #to avoid loading images
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
  
  driver = webdriver.Chrome('chromedriver',chrome_options=options)
  driver.get(url) 

  soup = BeautifulSoup(driver.page_source, 'lxml') 

  rawPageData = []
  for data in soup.find_all(['p','li']): 
    rawPageData.append(data.get_text())

  #pageData = list(set(rawPageData))
  
  return rawPageData #pageData

#### Pre-processing the scraped data

In [ ]:
%%capture
!pip install nltk

In [ ]:
%%capture
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.corpus import brown
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')
#from nltk.corpus import punkt


In [ ]:
def minPreProcess(inputData):
  data = []
  [data.append(x) for x in inputData if x not in data]

  clData = []
  for element in data:
    if element.count(" ") > 1:
      clData.append(element)
  
  return clData

#### Text Classification

In [ ]:
%%capture
!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
%%capture
#Required Libraries

#import pandas as pd
#import re
import numpy as np
#import torch
import transformers
from datasets import load_dataset, Dataset, load_from_disk
import string
import os
from google.colab import drive
#import csv

from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
#from transformers import DataCollatorWithPadding
#from transformers import create_optimizer
from datasets import load_metric
#import datasets
#from transformers.keras_callbacks import KerasMetricCallback

In [ ]:
def display(result):
  id2Label = {0: 'Data Retention', 1: 'Data Security', 2: 'Do Not Track', 3: 'First Party Collection/Use',
                     4: 'International and Specific Audiences', 5: 'Introductory/Generic', 6: 'Policy Change',
                     7: 'Practice not covered', 8:'Privacy contact information', 9: 'Third Party Sharing/Collection',
                     10: 'User Access, Edit and Deletion', 11: 'User Choice/Control'}
  
  joinedResult= dict()

  for key in result.keys():
    
    print(str(id2Label[key]) + "\n")
    print('\n'.join(map(str, result[key])))

    temp = ''.join(map(str, result[key]))
    joinedResult[id2Label[key]] = temp
    print("\n")
    
  return joinedResult

In [ ]:
def customizer(select):
  userOptions = dict()
  userOptions["old male"] = [6,3,9,11,1]
  userOptions["young male"] = [7,1,4,5]
  userOptions["young female"] = [7,6,1,4]
  userOptions["old female"] = [9,3,9,11,1]

  return userOptions[select]

In [ ]:
def classificationModel(datatoClassify, userType):
  # retreive the saved model 
  model_dir = '/content/drive/My Drive/Colab Notebooks/models/'
  num_labels = 12
    
  checkpoint = "mukund/privbert"
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  # For cleaner label outputs
  id2Label = {0: 'Data Retention', 1: 'Data Security', 2: 'Do Not Track', 3: 'First Party Collection/Use',
              4: 'International and Specific Audiences', 5: 'Introductory/Generic', 6: 'Policy Change',
              7: 'Practice not covered', 8:'Privacy contact information', 9: 'Third Party Sharing/Collection',
              10: 'User Access, Edit and Deletion', 11: 'User Choice/Control'}
             
  #saving the index of each label
  label2id = {val: key for key, val in id2Label.items()}

  '''the sequence classification head of the tranformer library is used to import the PrivBERT model 
    with mappings of the label'''
  model = TFAutoModelForSequenceClassification.from_pretrained(
      model_dir + 'pi', num_labels=num_labels, id2label=id2Label, label2id=label2id, local_files_only=True)
  
  user = customizer(userType)
  #to save the output
  result = dict()
  
  for i in range(len(datatoClassify)) :

    #print(datatoClassify[i])
    
    tokenized = tokenizer(str(datatoClassify[i]), return_tensors="np", padding="longest")
    
    # outputs = model(tokenized).logits
    outputs = model(tokenized).logits
    classificationsEncoded = np.argmax(outputs, axis=1)

    #print(classificationsEncoded)
    classifications = [model.config.id2label[output] for output in classificationsEncoded]
    
    #print(classifications)
    
    if int(classificationsEncoded) in user:
      
      if int(classificationsEncoded) not in result.keys():
       result[int(classificationsEncoded)]= []
      
      result[int(classificationsEncoded)].append(datatoClassify[i])
  
  return result 

#### Segment Summarization

In [ ]:
%%capture
#import nltk
#from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
#import numpy as np
import networkx as nx
#import pandas as pd
#from google.colab import drive
nltk.download('stopwords')

In [ ]:
def segmentToTokens(segments):
  temp2 = [word_tokenize(t) for t in sent_tokenize(segments)]
  return temp2

In [ ]:
def sentenceSimilarity(s1, s2, stopWords=None):
  if stopWords is None:
    stopWords = []
  s1 = [w.lower() for w in s1]    
  s2 = [w.lower() for w in s2]     
  wordList = list(set(s1 + s2))    
  vec1 = [0] * len(wordList)    
  vec2 = [0] * len(wordList)     
  
  # build the vector for the first sentence   
  for word in s1:
    if word in stopWords:
      continue        
    vec1[wordList.index(word)] += 1     

# build the vector for the second sentence    
  for w in s2:        
    if w in stopWords:           
      continue        
    vec2[wordList.index(w)] += 1     
  
  return 1 - cosine_distance(vec1,  vec2)

In [ ]:
def similarityMatrix(sentences, stopWords):
  similarityMtrx = np.zeros((len(sentences), len(sentences)))

  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if not i == j:
        similarityMtrx[i][j] = sentenceSimilarity(sentences[i], sentences[j], stopWords)
  return similarityMtrx

In [ ]:
def summarizer(sentences, length):
  stopWords = stopwords.words('english')
  summarizedText = []
  similarityMtrx = similarityMatrix(sentences, stopWords)
  similarityGraph = nx.from_numpy_array(similarityMtrx)
  scores = nx.pagerank(similarityGraph)
  ranks = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  print("Indexes of top rank order are ", ranks)
  for i in range(length):
    if ranks[i][1] != ranks[(i+1)][1]:
      summarizedText.append(" ".join(ranks[i][1]))
  summary = " ".join(summarizedText)
  return summary

In [ ]:
def displayOutput(opt, user):
  id2Label = {0: 'Data Retention', 1: 'Data Security', 2: 'Do Not Track', 3: 'First Party Collection/Use',
              4: 'International and Specific Audiences', 5: 'Introductory/Generic', 6: 'Policy Change',
              7: 'Practice not covered', 8:'Privacy contact information', 9: 'Third Party Sharing/Collection',
              10: 'User Access, Edit and Deletion', 11: 'User Choice/Control'}

  print(user.upper() + "\n")

  userType = customizer(user)

  for key in opt.keys():
    print(key)
    print(opt[key].replace('/\s+/',' ',).strip())
    print("\n")
  for k1 in userType:
    if id2Label[k1] not in opt.keys():
      print(id2Label[k1] + "\nNo data present.\n")



In [ ]:
def countWords(stage, content):
  count = 0
  if stage == "ini":
    for i in content:
      count = count + len(i.split())
  else :  
    for key in content.keys():
      count = count + len(opt[key].split())
  
  return count

#### Main

In [ ]:
#please input a user type  among : "young male", "old male", "young female", "old female"
user = "old male"

#please enter the url of the privacy policy
#url = "https://huggingface.co/privacy"
#url = "https://stackoverflow.com/legal/privacy-policy"
#url = "https://policies.google.com/privacy?"
#url = "https://www.linkedin.com/legal/privacy-policy" nw 
#url = "https://meta.wikimedia.org/wiki/Privacy_policy"
#url = "https://www.apple.com/legal/privacy/en-ww/" nw
#url = "https://www.samsung.com/au/info/privacy/" nw
#url = "https://legal.yahoo.com/au/en/yahoo/privacy/products/searchservices/index.html"
url = "https://docs.github.com/en/site-policy/privacy-policies/github-privacy-statement"

In [ ]:
rawData = scrapeUrl(url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [ ]:
rawData

In [ ]:
dataForClassification = minPreProcess(rawData)

In [ ]:
dataForClassification

In [ ]:
dataAfterClassification = dict()
dataAfterClassification = classificationModel(dataForClassification, user)

In [ ]:
joinedData = display(dataAfterClassification)

In [ ]:
opt = dict()
for key in joinedData.keys():
  seg = segmentToTokens(joinedData[key])
  if len(seg) > 5:
    opt[key] = summarizer(seg,5)
  else :
    opt[key] = joinedData[key]


Indexes of top rank order are  [(0.030795350483541328, ['For', 'example', ',', 'our', 'service', 'providers', 'may', 'perform', 'payment', 'processing', ',', 'customer', 'support', 'ticketing', ',', 'network', 'data', 'transmission', ',', 'web', 'analytics', ',', 'marketing', 'operations', ',', 'security', ',', 'and', 'other', 'similar', 'services', '.']), (0.03056572037826964, ['We', 'enable', 'access', 'to', 'personal', 'data', 'across', 'our', 'subsidiaries', ',', 'affiliates', ',', 'and', 'related', 'companies', ',', 'for', 'example', ',', 'where', 'we', 'share', 'common', 'data', 'systems', ',', 'when', 'affiliates', 'provide', 'services', 'on', 'our', 'behalf', ',', 'or', 'where', 'access', 'is', 'needed', 'to', 'operate', 'and', 'provide', 'the', 'Service', '.']), (0.030444423631045908, ['For', 'example', ',', 'this', 'may', 'happen', 'if', 'you', 'sign', 'up', 'for', 'training', 'or', 'to', 'receive', 'information', 'about', 'GitHub', 'from', 'one', 'of', 'our', 'vendors', ',',

In [ ]:
joinedData.keys()

dict_keys(['Third Party Sharing/Collection', 'Data Security', 'First Party Collection/Use', 'Policy Change', 'User Choice/Control'])

In [ ]:
displayOutput(opt, user)

OLD MALE

Third Party Sharing/Collection
For example , our service providers may perform payment processing , customer support ticketing , network data transmission , web analytics , marketing operations , security , and other similar services . We enable access to personal data across our subsidiaries , affiliates , and related companies , for example , where we share common data systems , when affiliates provide services on our behalf , or where access is needed to operate and provide the Service . For example , this may happen if you sign up for training or to receive information about GitHub from one of our vendors , partners , or affiliates . For example , our analytics and advertising partners may use these technologies in our Services to collect personal information ( such as the pages you visit , the links you click on , and similar usage information , identifiers , and device information ) related to your online activities over time and across Services for various purposes , i

In [ ]:
initialCount = countWords("ini", rawData)
finalCount = countWords("", opt)
print("Initial count = " + str(initialCount))
print("\nFinal count = " + str(finalCount))

Initial count = 6892

Final count = 1411
